## Ordering Basemaps via Orders API

In [11]:
import planet
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import time
import pathlib


### Ordering a Basemap via an AOI  

In [12]:
API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
orders_url = 'https://api.planet.com/compute/ops/orders/v2' 
auth = HTTPBasicAuth(API_KEY, '')
    
# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

In [13]:
response = requests.get(orders_url, auth=session.auth)
response

<Response [200]>

In [14]:
order_params = {
   "name": "basemap order with geometry",
   "source_type": "basemaps",
   "order_type":"partial",
   "products": [
       {
           "mosaic_name": "global_monthly_2022_01_mosaic",
           "geometry":{
               "type": "Polygon",
               "coordinates":[
                   [
                       [ 4.607406, 52.353994 ],
                        [ 4.680005, 52.353994 ],
                        [ 4.680005, 52.395523 ],
                        [ 4.607406, 52.395523 ],
                        [ 4.607406, 52.353994 ]
                   ]
               ]
           }
      }
  ],
    "tools": [
      {"merge": {}},
      {"clip": {}}
      

  ],
 "delivery":{
      "google_cloud_storage":{
         "bucket":"notebook-practice",
         "credentials":GCP_CREDENTIALS,
         "path_prefix":"basemaps-to-cloud/"
      }


 }
}



In [15]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
headers = {'content-type': 'application/json'}
paramRes = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers = {'content-type': 'application/json'})

print(paramRes.text)

{"code":602,"message":"credentials in body is required"}


In [16]:
def place_order(order_params, auth):
    response = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url

In [17]:
order_url = place_order(order_params, session.auth)

<Response [400]>


KeyError: 'id'

In [ ]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
       
        
poll_for_success(order_url, auth)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/c33d0332-0bf0-4eb6-95fd-a2715d065300'}, 'created_on': '2022-11-30T21:46:29.114Z', 'delivery': {'google_cloud_storage': {'bucket': 'notebook-practice', 'credentials': '<REDACTED>', 'path_prefix': 'basemaps-to-cloud/'}}, 'error_hints': [], 'id': 'c33d0332-0bf0-4eb6-95fd-a2715d065300', 'last_message': 'Preparing order', 'last_modified': '2022-11-30T21:46:29.114Z', 'name': 'basemap order with geometry', 'order_type': 'partial', 'products': [{'geometry': {'coordinates': [[[4.607406, 52.353994], [4.680005, 52.353994], [4.680005, 52.395523], [4.607406, 52.395523], [4.607406, 52.353994]]], 'type': 'Polygon'}, 'mosaic_name': 'global_monthly_2022_01_mosaic'}], 'source_type': 'basemaps', 'state': 'queued', 'tools': [{'merge': {}}, {'clip': {}}]}
queued
{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/c33d0332-0bf0-4eb6-95fd-a2715d065300'}, 'created_on': '2022-11-30T21:46:29.114Z', 'delivery': {'google_cloud_stora

In [ ]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

KeyError: 'results'

In [ ]:
[r['name'] for r in results]

['basemaps-to-cloud/949f788f-7639-4d45-a6cf-06bbdda814a9/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_metadata_merge_clip.json',
 'basemaps-to-cloud/949f788f-7639-4d45-a6cf-06bbdda814a9/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_provenance_raster_merge_clip.tif',
 'basemaps-to-cloud/949f788f-7639-4d45-a6cf-06bbdda814a9/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_provenance_vector_merge_clip.zip',
 'basemaps-to-cloud/949f788f-7639-4d45-a6cf-06bbdda814a9/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_merge_clip.tif',
 'basemaps-to-cloud/949f788f-7639-4d45-a6cf-06bbdda814a9/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_ortho_udm_merge_clip.tif',
 'basemaps-to-cloud/949f788f-7639-4d45-a6cf-06bbdda814a9/manifest.json']